# 🦠 HBV scRNA-seq 데이터 다운로드

**목적**: GEO에서 HBV 관련 scRNA-seq 데이터 다운로드 및 전처리

---

## 📊 대상 데이터셋: GSE182159

| 항목 | 내용 |
|------|------|
| 제목 | HBV 감염 환자의 간/혈액 면역 특성 |
| 세포 수 | 243,000개 |
| 샘플 | 46개 (23명, 간+PBMC 쌍) |
| 그룹 | 건강, 급성 HBV, 만성 HBV, 완치 |

---
## STEP 1: 환경 설정

In [ ]:
# @title STEP 1: 환경 설정
import os, sys

# Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 경로 설정
PROJECT_PATH = '/content/drive/MyDrive/tahoe-x1'
DATA_PATH = f'{PROJECT_PATH}/viral_hepatitis/raw_data'
os.makedirs(DATA_PATH, exist_ok=True)
os.chdir(DATA_PATH)

# 패키지 설치
print('📦 패키지 설치 중...')
!pip install GEOparse scanpy anndata -q

print(f'\n✅ STEP 1 완료!')
print(f'📁 데이터 저장 경로: {DATA_PATH}')

---
## STEP 2: GSE182159 메타데이터 확인

In [ ]:
# @title STEP 2: GSE182159 메타데이터 확인
import GEOparse

print('🔍 GSE182159 메타데이터 다운로드 중...')
gse = GEOparse.get_GEO(geo='GSE182159', destdir='./')

print('\n📋 데이터셋 정보:')
print(f'   제목: {gse.metadata["title"][0]}')
print(f'   플랫폼: {list(gse.gpls.keys())}')
print(f'   샘플 수: {len(gse.gsms)}')

print('\n📊 샘플 목록 (처음 10개):')
for i, (gsm_id, gsm) in enumerate(list(gse.gsms.items())[:10]):
    print(f'   {gsm_id}: {gsm.metadata["title"][0]}')

---
## STEP 3: 보조 파일 다운로드

⚠️ **주의**: 대용량 파일 (수 GB) - 시간이 걸릴 수 있습니다.

In [ ]:
# @title STEP 3: 보조 파일 다운로드 (처리된 데이터)
import urllib.request
import os

# GSE182159 supplementary files
# 보통 처리된 카운트 매트릭스가 포함됨
base_url = 'https://ftp.ncbi.nlm.nih.gov/geo/series/GSE182nnn/GSE182159/suppl/'

# 파일 목록 확인
print('📥 보조 파일 확인 중...')
print(f'   URL: {base_url}')
print('\n💡 수동 다운로드 방법:')
print('   1. 위 URL을 브라우저에서 열기')
print('   2. 필요한 파일 다운로드')
print('   3. raw_data 폴더에 업로드')

# 자동 다운로드 시도 (처리된 데이터)
print('\n🔄 자동 다운로드 시도...')
!wget -q --show-progress -P ./ \
  "https://ftp.ncbi.nlm.nih.gov/geo/series/GSE182nnn/GSE182159/suppl/GSE182159_RAW.tar" \
  2>/dev/null || echo '⚠️ RAW.tar 파일 없음 - 개별 파일 확인 필요'

---
## STEP 4: 대안 - 처리된 데이터 다운로드

많은 연구들이 GitHub나 Zenodo에 처리된 h5ad 파일을 공개합니다.

In [ ]:
# @title STEP 4: 예제 HBV 데이터 생성 (테스트용)
# GSE182159 원본 데이터가 제한적인 경우, 
# PBMC 3k를 HBV 시뮬레이션 데이터로 사용

import scanpy as sc
import numpy as np

print('🧬 테스트용 HBV 시뮬레이션 데이터 생성...')

# PBMC 3k 로드
adata = sc.datasets.pbmc3k()
sc.pp.filter_genes(adata, min_cells=3)

# 가상의 HBV 상태 할당 (데모용)
np.random.seed(42)
n_cells = adata.n_obs
hbv_status = np.random.choice(
    ['Healthy', 'Chronic_HBV_Tolerant', 'Chronic_HBV_Active', 'Resolved_HBV'],
    size=n_cells,
    p=[0.25, 0.25, 0.25, 0.25]
)
adata.obs['hbv_status'] = hbv_status

# 저장
output_file = f'{DATA_PATH}/simulated_hbv_pbmc.h5ad'
adata.write_h5ad(output_file)

print(f'\n✅ 테스트 데이터 생성 완료!')
print(f'   파일: {output_file}')
print(f'   세포 수: {adata.n_obs}')
print(f'\n📊 HBV 상태별 세포 수:')
print(adata.obs['hbv_status'].value_counts())

---
## STEP 5: 데이터 확인

In [ ]:
# @title STEP 5: 다운로드된 파일 확인
import os

print('📁 raw_data 폴더 내용:')
for f in os.listdir(DATA_PATH):
    size = os.path.getsize(os.path.join(DATA_PATH, f)) / 1e6
    print(f'   {f} ({size:.1f} MB)')

print('\n✅ STEP 5 완료!')

---
## 📚 다음 단계

1. **실제 데이터 획득**: GEO 원저자에게 연락하거나 공개된 처리 데이터 찾기
2. **Tahoe-x1 분석**: `PBMC_inference_26Jan12.ipynb` 파이프라인 적용
3. **그룹 비교**: 건강인 vs HBV 환자 면역세포 분포 비교

### 📧 데이터 요청
GSE182159 원본 데이터는 환자 개인정보 보호로 일부 제한될 수 있습니다.  
논문 저자에게 연락하여 처리된 데이터를 요청하는 것이 좋습니다.